In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("./dog_breeds/all/"))

# Any results you write to the current directory are saved as output.

## Initial Imports

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import shutil
from glob import glob 

from sklearn.utils import shuffle 
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

## Specify a GPU
import os

Using TensorFlow backend.


In [2]:
base_dir = '../input/'
my_dir = '../input/dog_breeds/'
train_folder = os.path.join(my_dir,'train')
valid_folder = os.path.join(my_dir,'valid')
test_folder = os.path.join(my_dir,'test')

# train_folder = os.path.join(base_dir,'base_dir/train')
# valid_folder = os.path.join(base_dir,'base_dir/valid')
# test_folder = os.path.join(base_dir,'base_dir/test')
val_file, train_file, test_file= [os.path.join("../input/dog-breeds", name) 
                                          for name in os.listdir("../input/dog-breeds/")]

In [3]:
df_train = pd.read_csv(train_file)
df_val = pd.read_csv(val_file)

In [4]:
df_test = pd.read_csv(test_file)
# df_data = df.merge(labels, on = "id")
df_test.head()

,breed_id,breed,id
0,n02094114,Norfolk_terrier,n02094114_2449.jpg
1,n02106030,collie,n02106030_16181.jpg
2,n02089078,black,n02089078_4497.jpg
3,n02098286,West_Highland_white_terrier,n02098286_2863.jpg
4,n02105505,komondor,n02105505_3030.jpg


## Only run for the first time: split training data into training and validation

In [5]:
os.makedirs(os.path.join(my_dir, "all/"))

In [6]:
train_y = df_train['breed']

train_folder = os.path.join(my_dir,'train')
valid_folder = os.path.join(my_dir,'valid')
test_folder = os.path.join(my_dir,'test')
for fold in [train_folder, valid_folder, test_folder]:
    for subf in train_y.unique():
        os.makedirs(os.path.join(fold, subf))

In [7]:
origin_data_dir =  os.path.join(my_dir, "all/")
df_train.set_index('id', inplace=True)
df_val.set_index('id', inplace=True)
df_test.set_index('id', inplace=True)

In [8]:
origin_path = '../input/stanford-dogs-dataset/images/Images/'
for breed in os.listdir(origin_path):
    for file in glob(os.path.join(origin_path,breed,'*.jpg')):
        shutil.copyfile(file, os.path.join(origin_data_dir, file.split('/')[-1:][0]))

In [9]:
for image in df_train.index.values:
    breed = str(df_train.loc[image,'breed']) # get the label for a certain image
    src = os.path.join(origin_data_dir, image)
    dst = os.path.join(train_folder, breed, image)
    shutil.copyfile(src, dst)

In [10]:
for image in df_test.index.values:
    breed = str(df_test.loc[image,'breed']) # get the label for a certain image
    src = os.path.join(origin_data_dir, image)
    dst = os.path.join(test_folder, breed, image)
    shutil.copyfile(src, dst)
for image in df_val.index.values:
    breed = str(df_val.loc[image,'breed']) # get the label for a certain image
    src = os.path.join(origin_data_dir, image)
    dst = os.path.join(valid_folder, breed, image)
    shutil.copyfile(src, dst)

## Image Data Preprocessing

In [31]:
IMAGE_SIZE = 200
num_train_samples = len(df_train)
num_val_samples = len(df_val)
num_test_samples = len(df_test)

train_batch_size = 32
val_batch_size = 32
test_batch_size = 32
print("Num of train samples: %d" % num_train_samples)
print("Num of validation samples: %d" % num_val_samples)
print("Num of test samples: %d" % num_test_samples)

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)
test_steps = np.ceil(num_test_samples / test_batch_size)
print("train_steps: %d" % train_steps)
print("val_steps: %d" % val_steps)
print("test_steps: %d" % test_steps)

Num of train samples: 4116
Num of validation samples: 3293
Num of test samples: 13171
train_steps: 129
val_steps: 103
test_steps: 412


In [32]:
from keras.preprocessing.image import ImageDataGenerator
## Noramlize RGB values
datagen = ImageDataGenerator(
    rescale=1./255,
    # preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x,
    horizontal_flip=True,
    vertical_flip=True)

In [33]:
train_gen = datagen.flow_from_directory(train_folder,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(valid_folder,
                                      target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                      batch_size=val_batch_size,
                                      class_mode='categorical')

test_gen = datagen.flow_from_directory(test_folder,
                                       target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                       batch_size=test_batch_size,
                                       class_mode='categorical',
                                       shuffle=False)

Found 4116 images belonging to 120 classes.
Found 3293 images belonging to 120 classes.
Found 13171 images belonging to 120 classes.


## InceptionResNetV2 Model

In [27]:
## InceptionResNetV2
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Sequential, Model
from keras import layers
from keras import optimizers
from keras.layers import Concatenate, Activation, BatchNormalization, Flatten, Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D

IMG_SIZE = (IMAGE_SIZE, IMAGE_SIZE)
IN_SHAPE = (*IMG_SIZE, 3)

dropout_dense=0.5

conv_base = InceptionResNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=IN_SHAPE
)

model = Sequential()
model.add(conv_base)
## Version1
# model.add(GlobalAveragePooling2D())
# model.add(layers.Dense(1024, activation = "relu"))
# model.add(layers.Dense(120, activation = "softmax"))

##Version 2 
# model.add(GlobalAveragePooling2D())
# model.add(layers.Dense(1024))
# model.add(BatchNormalization())
# model.add(Activation("relu"))
# model.add(layers.Dense(120, activation = "softmax"))


## Version 3
# model.add(BatchNormalization())
# model.add(Dropout(0.001))
# model.add(GlobalMaxPooling2D())
# #model.add(layers.Dense(120, activation = "sigmoid"))
# model.add(layers.Dense(512, activation = "relu"))
# model.add(BatchNormalization())
# model.add(Dropout(0.001))
# model.add(layers.Dense(120, activation = "softmax"))

## Version 4
#model.add(GlobalAveragePooling2D())
#model.add(GlobalMaxPooling2D())
model.add(Flatten())
model.add(Dropout(0.001))
model.add(layers.Dense(120, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.001))
model.add(layers.Dense(120, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.001))
model.add(layers.Dense(120, activation = "softmax"))

# conv_base.summary()
conv_base.Trainable=True

from keras import optimizers

# conv_base.trainable = False
model.compile(optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics=["accuracy"])
# model.load_weights('../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [28]:
## Train inception_resnet model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
h5_path = "inception_resnet_v2_relu_softmax_10.h5"
checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1,restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr = 0.000000001)
history = model.fit_generator(train_gen, steps_per_epoch=train_steps, 
                              validation_data=val_gen, validation_steps=val_steps,
                              epochs=20,
                              callbacks=[reducel, checkpoint])

Epoch 1/25
129/129 [==============================] - 232s 2s/step - loss: 3.3896 - acc: 0.3242 - val_loss: 2.0304 - val_acc: 0.6116

Epoch 00001: val_acc improved from -inf to 0.61160, saving model to inception_resnet_v2_relu_softmax_10.h5
Epoch 2/25
129/129 [==============================] - 120s 926ms/step - loss: 1.7131 - acc: 0.6712 - val_loss: 1.6571 - val_acc: 0.6496

Epoch 00002: val_acc improved from 0.61160 to 0.64956, saving model to inception_resnet_v2_relu_softmax_10.h5
Epoch 3/25
129/129 [==============================] - 119s 923ms/step - loss: 1.1017 - acc: 0.7843 - val_loss: 1.5193 - val_acc: 0.6881

Epoch 00003: val_acc improved from 0.64956 to 0.68813, saving model to inception_resnet_v2_relu_softmax_10.h5
Epoch 4/25
129/129 [==============================] - 119s 925ms/step - loss: 0.7819 - acc: 0.8542 - val_loss: 1.5125 - val_acc: 0.6760

Epoch 00004: val_acc did not improve from 0.68813
Epoch 5/25
129/129 [==============================] - 119s 923ms/step - loss: 

In [29]:
# accuracy metric 1
from keras.metrics import categorical_accuracy
result = model.evaluate_generator(test_gen,steps = test_steps)
print(model.metrics_names)
print(result)

['loss', 'acc']
[1.1206932214427652, 0.7601548857519109]


## Xception Model

In [18]:
## Xception
from keras.applications.xception import Xception
from keras.models import Sequential, Model
from keras import layers
from keras import optimizers
from keras.layers import Concatenate, BatchNormalization, Flatten, Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D

IMG_SIZE = (IMAGE_SIZE, IMAGE_SIZE)
IN_SHAPE = (*IMG_SIZE, 3)

conv_base = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=IN_SHAPE
)

model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(layers.Dense(1024, activation = "relu"))
model.add(layers.Dense(120, activation = "softmax")

# conv_base.summary()
conv_base.Trainable=True

from keras import optimizers

# conv_base.trainable = False
model.compile(optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics=["accuracy"])
# model.load_weights('../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [19]:
## Train inception_resnet model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
h5_path = "xception.h5"
checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1,restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr = 0.000000001)
history = model.fit_generator(train_gen, steps_per_epoch=train_steps, 
                              validation_data=val_gen, validation_steps=val_steps,
                              epochs=20,
                              callbacks=[reducel, checkpoint])

Epoch 1/20
129/129 [==============================] - 77s 599ms/step - loss: 1.9983 - acc: 0.5819 - val_loss: 1.5942 - val_acc: 0.5940

Epoch 00001: val_acc improved from -inf to 0.59399, saving model to xception.h5
Epoch 2/20
129/129 [==============================] - 60s 468ms/step - loss: 0.4172 - acc: 0.9139 - val_loss: 1.4777 - val_acc: 0.6213

Epoch 00002: val_acc improved from 0.59399 to 0.62132, saving model to xception.h5
Epoch 3/20
129/129 [==============================] - 61s 469ms/step - loss: 0.2132 - acc: 0.9555 - val_loss: 1.4880 - val_acc: 0.6383

Epoch 00003: val_acc improved from 0.62132 to 0.63832, saving model to xception.h5
Epoch 4/20
129/129 [==============================] - 61s 472ms/step - loss: 0.1370 - acc: 0.9734 - val_loss: 1.4914 - val_acc: 0.6408

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 00004: val_acc improved from 0.63832 to 0.64075, saving model to xception.h5
Epoch 5/20
129/129 [==========================

In [20]:
# accuracy metric 1
from keras.metrics import categorical_accuracy
result = model.evaluate_generator(test_gen,steps = test_steps)
print(model.metrics_names)
print(result)

['loss', 'acc']
[1.3929535331183744, 0.6614531926608793]


## ResNet50

In [46]:
## ResNet50
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential, Model
from keras import layers
from keras import optimizers
from keras.layers import Concatenate, BatchNormalization, Flatten, Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D

IMG_SIZE = (IMAGE_SIZE, IMAGE_SIZE)
IN_SHAPE = (*IMG_SIZE, 3)

conv_base = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=IN_SHAPE
)

model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(layers.Dense(1024, activation = "relu"))
model.add(layers.Dense(120, activation = "softmax"))

# conv_base.summary()
conv_base.Trainable=True

from keras import optimizers

# conv_base.trainable = False
model.compile(optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics=["accuracy"])
# model.load_weights('../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')

/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [47]:
## Train resnet50 model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
h5_path = "ResNet50.h5"
checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1,restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr = 0.000000001)
history = model.fit_generator(train_gen, steps_per_epoch=train_steps, 
                              validation_data=val_gen, validation_steps=val_steps,
                              epochs=10,
                              callbacks=[reducel, checkpoint])

Epoch 1/10
129/129 [==============================] - 150s 1s/step - loss: 3.9181 - acc: 0.1622 - val_loss: 3.3425 - val_acc: 0.2305

Epoch 00001: val_acc improved from -inf to 0.23049, saving model to ResNet50.h5
Epoch 2/10
129/129 [==============================] - 47s 364ms/step - loss: 2.0939 - acc: 0.4569 - val_loss: 2.3535 - val_acc: 0.3863

Epoch 00002: val_acc improved from 0.23049 to 0.38627, saving model to ResNet50.h5
Epoch 3/10
129/129 [==============================] - 47s 366ms/step - loss: 1.3109 - acc: 0.6444 - val_loss: 2.2176 - val_acc: 0.4197

Epoch 00003: val_acc improved from 0.38627 to 0.41968, saving model to ResNet50.h5
Epoch 4/10
129/129 [==============================] - 46s 359ms/step - loss: 0.7823 - acc: 0.7886 - val_loss: 2.1262 - val_acc: 0.4388

Epoch 00004: val_acc improved from 0.41968 to 0.43881, saving model to ResNet50.h5
Epoch 5/10
129/129 [==============================] - 46s 357ms/step - loss: 0.5053 - acc: 0.8760 - val_loss: 2.1993 - val_acc: 0

In [48]:
# accuracy metric 1
from keras.metrics import categorical_accuracy
result = model.evaluate_generator(test_gen,steps = test_steps)
print(model.metrics_names)
print(result)

['loss', 'acc']
[1.822889103631334, 0.5343557816052815]


## InceptionV3

In [36]:
## InceptionV3
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential, Model
from keras import layers
from keras import optimizers
from keras.layers import Concatenate, BatchNormalization, Flatten, Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D

IMG_SIZE = (IMAGE_SIZE, IMAGE_SIZE)
IN_SHAPE = (*IMG_SIZE, 3)

conv_base = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=IN_SHAPE
)

model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(layers.Dense(1024, activation = "relu"))
model.add(layers.Dense(120, activation = "softmax"))

# conv_base.summary()
conv_base.Trainable=True

from keras import optimizers

# conv_base.trainable = False
model.compile(optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics=["accuracy"])
# model.load_weights('../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')

87916544/87910968 [==============================] - 2s 0us/step


In [37]:
## Train inception_resnet model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
h5_path = "InceptionV3.h5"
checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1,restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr = 0.000000001)
history = model.fit_generator(train_gen, steps_per_epoch=train_steps, 
                              validation_data=val_gen, validation_steps=val_steps,
                              epochs=10,
                              callbacks=[reducel, checkpoint])

Epoch 1/10
129/129 [==============================] - 128s 989ms/step - loss: 3.9884 - acc: 0.1672 - val_loss: 2.8011 - val_acc: 0.3371

Epoch 00001: val_acc improved from -inf to 0.33708, saving model to InceptionV3.h5
Epoch 2/10
129/129 [==============================] - 41s 314ms/step - loss: 2.1775 - acc: 0.4565 - val_loss: 2.0992 - val_acc: 0.4446

Epoch 00002: val_acc improved from 0.33708 to 0.44458, saving model to InceptionV3.h5
Epoch 3/10
129/129 [==============================] - 41s 321ms/step - loss: 1.4280 - acc: 0.6071 - val_loss: 1.9280 - val_acc: 0.4719

Epoch 00003: val_acc improved from 0.44458 to 0.47191, saving model to InceptionV3.h5
Epoch 4/10
129/129 [==============================] - 41s 318ms/step - loss: 0.9760 - acc: 0.7259 - val_loss: 1.7866 - val_acc: 0.5141

Epoch 00004: val_acc improved from 0.47191 to 0.51412, saving model to InceptionV3.h5
Epoch 5/10
129/129 [==============================] - 43s 330ms/step - loss: 0.6936 - acc: 0.8045 - val_loss: 1.96

In [38]:
# accuracy metric 1
from keras.metrics import categorical_accuracy
result = model.evaluate_generator(test_gen,steps = test_steps)
print(model.metrics_names)
print(result)

['loss', 'acc']
[1.5274156829543486, 0.6010933110259785]
